models : decision trees / Naive Bayes / KNN

In [1]:
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv('Flight_delay.csv')
df.head()

,DayOfWeek,Date,DepTime,ArrTime,CRSArrTime,UniqueCarrier,Airline,FlightNum,TailNum,ActualElapsedTime,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,4,03-01-2019,1829,1959,1925,WN,Southwest Airlines Co.,3920,N464WN,90,...,3,10,0,N,0,2,0,0,0,32
1,4,03-01-2019,1937,2037,1940,WN,Southwest Airlines Co.,509,N763SW,240,...,3,7,0,N,0,10,0,0,0,47
2,4,03-01-2019,1644,1845,1725,WN,Southwest Airlines Co.,1333,N334SW,121,...,6,8,0,N,0,8,0,0,0,72
3,4,03-01-2019,1452,1640,1625,WN,Southwest Airlines Co.,675,N286WN,228,...,7,8,0,N,0,3,0,0,0,12
4,4,03-01-2019,1323,1526,1510,WN,Southwest Airlines Co.,4,N674AA,123,...,4,9,0,N,0,0,0,0,0,16


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484551 entries, 0 to 484550
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DayOfWeek          484551 non-null  int64 
 1   Date               484551 non-null  object
 2   DepTime            484551 non-null  int64 
 3   ArrTime            484551 non-null  int64 
 4   CRSArrTime         484551 non-null  int64 
 5   UniqueCarrier      484551 non-null  object
 6   Airline            484551 non-null  object
 7   FlightNum          484551 non-null  int64 
 8   TailNum            484551 non-null  object
 9   ActualElapsedTime  484551 non-null  int64 
 10  CRSElapsedTime     484551 non-null  int64 
 11  AirTime            484551 non-null  int64 
 12  ArrDelay           484551 non-null  int64 
 13  DepDelay           484551 non-null  int64 
 14  Origin             484551 non-null  object
 15  Org_Airport        483374 non-null  object
 16  Dest               4

In [4]:
df = df[['DayOfWeek','Date','DepTime','ArrTime','CRSArrTime','Airline','Origin','Dest','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']]
df

,DayOfWeek,Date,DepTime,ArrTime,CRSArrTime,Airline,Origin,Dest,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,4,03-01-2019,1829,1959,1925,Southwest Airlines Co.,IND,BWI,2,0,0,0,32
1,4,03-01-2019,1937,2037,1940,Southwest Airlines Co.,IND,LAS,10,0,0,0,47
2,4,03-01-2019,1644,1845,1725,Southwest Airlines Co.,IND,MCO,8,0,0,0,72
3,4,03-01-2019,1452,1640,1625,Southwest Airlines Co.,IND,PHX,3,0,0,0,12
4,4,03-01-2019,1323,1526,1510,Southwest Airlines Co.,IND,TPA,0,0,0,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
484546,5,13-06-2019,1609,1942,1915,American Airlines Inc.,DFW,MCO,27,0,0,0,0
484547,6,14-06-2019,1616,1954,1915,American Airlines Inc.,DFW,MCO,11,0,0,0,28
484548,2,17-06-2019,1617,2002,1915,American Airlines Inc.,DFW,MCO,0,22,5,0,20
484549,7,22-06-2019,1607,1941,1915,American Airlines Inc.,DFW,MCO,1,0,0,0,25


In [5]:
delay_cols = [
    "CarrierDelay",
    "WeatherDelay",
    "NASDelay",
    "SecurityDelay",
    "LateAircraftDelay"
]
df["Delay"] = (df[delay_cols].sum(axis=1) > 30).astype(int)
df = df.drop(columns=delay_cols)
df.head()


,DayOfWeek,Date,DepTime,ArrTime,CRSArrTime,Airline,Origin,Dest,Delay
0,4,03-01-2019,1829,1959,1925,Southwest Airlines Co.,IND,BWI,1
1,4,03-01-2019,1937,2037,1940,Southwest Airlines Co.,IND,LAS,1
2,4,03-01-2019,1644,1845,1725,Southwest Airlines Co.,IND,MCO,1
3,4,03-01-2019,1452,1640,1625,Southwest Airlines Co.,IND,PHX,0
4,4,03-01-2019,1323,1526,1510,Southwest Airlines Co.,IND,TPA,0


In [6]:
df.isnull().sum()

,0
DayOfWeek,0
Date,0
DepTime,0
ArrTime,0
CRSArrTime,0
Airline,0
Origin,0
Dest,0
Delay,0


In [7]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

In [8]:
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

In [9]:
df = df.drop(columns=['Date'])

In [10]:
df.head()

,DayOfWeek,DepTime,ArrTime,CRSArrTime,Airline,Origin,Dest,Delay,month,day
0,4,1829,1959,1925,Southwest Airlines Co.,IND,BWI,1,1,3
1,4,1937,2037,1940,Southwest Airlines Co.,IND,LAS,1,1,3
2,4,1644,1845,1725,Southwest Airlines Co.,IND,MCO,1,1,3
3,4,1452,1640,1625,Southwest Airlines Co.,IND,PHX,0,1,3
4,4,1323,1526,1510,Southwest Airlines Co.,IND,TPA,0,1,3


In [11]:
categorical_features = df.select_dtypes(include=['object']).columns
categorical_features

Index(['Airline', 'Origin', 'Dest'], dtype='object')

In [12]:
for col in categorical_features:
    print(f"\nColumn: {col}")
    print(df[col].value_counts())


Column: Airline
Airline
Southwest Airlines Co.          119048
American Airlines Inc.           73053
American Eagle Airlines Inc.     58698
United Air Lines Inc.            56896
Skywest Airlines Inc.            50384
US Airways Inc.                  31755
Delta Air Lines Inc.             30220
Atlantic Southeast Airlines      28678
JetBlue Airways                  15364
Alaska Airlines Inc.             10000
Frontier Airlines Inc.            9015
Hawaiian Airlines Inc.            1440
Name: count, dtype: int64

Column: Origin
Origin
ORD    46945
DFW    33027
ATL    28834
DEN    23543
LAX    17194
       ...  
DLG        9
AKN        9
ADK        7
BLI        2
WYS        1
Name: count, Length: 274, dtype: int64

Column: Dest
Dest
ORD    40622
DFW    24543
ATL    23557
DEN    19250
LAX    18350
       ...  
ADK       10
DLG        9
BLI        7
AKN        7
WYS        5
Name: count, Length: 274, dtype: int64


In [13]:
df_encoded = pd.get_dummies(df, dtype = int)

In [14]:
df_encoded.head()

,DayOfWeek,DepTime,ArrTime,CRSArrTime,Delay,month,day,Airline_Alaska Airlines Inc.,Airline_American Airlines Inc.,Airline_American Eagle Airlines Inc.,...,Dest_TYR,Dest_TYS,Dest_VLD,Dest_VPS,Dest_WRG,Dest_WYS,Dest_XNA,Dest_YAK,Dest_YKM,Dest_YUM
0,4,1829,1959,1925,1,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,1937,2037,1940,1,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,1644,1845,1725,1,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1452,1640,1625,0,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1323,1526,1510,0,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_encoded.head()

,DayOfWeek,DepTime,ArrTime,CRSArrTime,Delay,month,day,Airline_Alaska Airlines Inc.,Airline_American Airlines Inc.,Airline_American Eagle Airlines Inc.,...,Dest_TYR,Dest_TYS,Dest_VLD,Dest_VPS,Dest_WRG,Dest_WYS,Dest_XNA,Dest_YAK,Dest_YKM,Dest_YUM
0,4,1829,1959,1925,1,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,1937,2037,1940,1,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,1644,1845,1725,1,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1452,1640,1625,0,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1323,1526,1510,0,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
from sklearn.preprocessing import MinMaxScaler
numerical_features = df_encoded.select_dtypes(include=['int64', 'float64']).columns
scaler = MinMaxScaler()
df_encoded[numerical_features] = scaler.fit_transform(df_encoded[numerical_features])


print(df_encoded.head())


   DayOfWeek   DepTime   ArrTime  CRSArrTime  Delay  month  day  \
0        0.5  0.761984  0.816173    0.815946    1.0      1    3   
1        0.5  0.807003  0.848687    0.822307    1.0      1    3   
2        0.5  0.684869  0.768654    0.731128    1.0      1    3   
3        0.5  0.604835  0.683201    0.688719    0.0      1    3   
4        0.5  0.551063  0.635682    0.639949    0.0      1    3   

   Airline_Alaska Airlines Inc.  Airline_American Airlines Inc.  \
0                           0.0                             0.0   
1                           0.0                             0.0   
2                           0.0                             0.0   
3                           0.0                             0.0   
4                           0.0                             0.0   

   Airline_American Eagle Airlines Inc.  ...  Dest_TYR  Dest_TYS  Dest_VLD  \
0                                   0.0  ...       0.0       0.0       0.0   
1                                   0.

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [18]:
X = df_encoded.drop(columns=['Delay'])
y = df_encoded['Delay']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [20]:
print("X=",X_train.shape, X_test.shape)
print("y=",y_train.shape, y_test.shape)

X= (339185, 566) (145366, 566)
y= (339185,) (145366,)


In [21]:
clf = DecisionTreeClassifier(random_state=1)



In [22]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=1)

In [23]:
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9946892670913419


## Export Model for Web Application

Save the trained model, scaler, and feature columns for use in the Flask API

In [ ]:
import pickle

# Save the trained Decision Tree model
with open('flight_delay_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

# Save the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save the feature columns (needed for one-hot encoding alignment)
with open('feature_columns.pkl', 'wb') as f:
    pickle.dump(X.columns.tolist(), f)

# Save the unique airlines and airports from training data
airlines_list = df['Airline'].unique().tolist()
origins_list = df['Origin'].unique().tolist()
dests_list = df['Dest'].unique().tolist()

model_info = {
    'airlines': airlines_list,
    'origins': origins_list,
    'destinations': dests_list
}

with open('model_info.pkl', 'wb') as f:
    pickle.dump(model_info, f)

print("Model and preprocessing objects saved successfully!")
print(f"Accuracy: {accuracy}")
print(f"Total features: {len(X.columns)}")

In [24]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb = GaussianNB()


gnb.fit(X_train, y_train)


y_pred = gnb.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')